In [1]:
import os
import sys
import math
import random

import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.spatial import distance
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, cophenet
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(0, '../../')
import ccal
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

==================== Computational Cancer Analysis Library ====================
<15:24:02.489384> Checking dependencies ...
<15:24:02.491521> Using the following packages:
<15:24:02.493184> 	matplotlib (v1.5.1)
<15:24:02.493201> 	numpy (v1.10.4)
<15:24:02.493211> 	pandas (v0.18.0)
<15:24:02.493231> 	rpy2 (v2.7.9)
<15:24:02.493240> 	scikit-learn (v0.17.1)
<15:24:02.493247> 	scipy (v0.17.0)
<15:24:02.493254> 	seaborn (v0.7.0)


In [2]:
features = ccal.support.read_gct('/Users/Kwat/Downloads/v22_pub.gct')

component_x_cellline = ccal.support.read_gct('/Users/Kwat/Downloads/CCLE.rpkm.v2.SELECTED_SIGNATURES.v2.gct')
ref = component_x_cellline.ix['KRAS_SALE_Late_Comp_C1_9', :]

In [3]:
features

,CHP212_AUTONOMIC_GANGLIA,IMR32_AUTONOMIC_GANGLIA,KELLY_AUTONOMIC_GANGLIA,KPNSI9S_AUTONOMIC_GANGLIA,KPNYN_AUTONOMIC_GANGLIA,MHHNB11_AUTONOMIC_GANGLIA,NB1_AUTONOMIC_GANGLIA,NH6_AUTONOMIC_GANGLIA,SIMA_AUTONOMIC_GANGLIA,SKNAS_AUTONOMIC_GANGLIA,...,RT112_URINARY_TRACT,RT4_URINARY_TRACT,SCABER_URINARY_TRACT,AM38_CENTRAL_NERVOUS_SYSTEM,T24_URINARY_TRACT,TCCSUP_URINARY_TRACT,UBLC1_URINARY_TRACT,UMUC1_URINARY_TRACT,UMUC3_URINARY_TRACT,VMCUB1_URINARY_TRACT
ML311,11.0250,9.35600,10.7340,11.3370,14.0640,8.4827,11.1810,12.0160,NaN,12.8430,...,12.06600,11.9200,12.2380,7.7821,12.6020,13.4000,11.6220,12.4470,11.2840,11.9120
zebularine,14.0780,16.16900,15.0000,13.8280,14.6360,NaN,14.2480,15.0930,NaN,14.7040,...,14.80200,15.7820,14.7190,NaN,14.7260,14.9850,15.9120,14.8360,13.6360,14.3570
BRD-A02303741,NaN,14.65700,15.0260,15.0720,14.4010,13.7550,14.7160,14.5990,NaN,NaN,...,15.33900,15.1450,16.0060,15.0000,14.2980,15.0000,15.8190,14.0940,15.2230,14.5720
importazole,12.2130,14.18300,13.3760,14.8660,15.0540,NaN,14.5440,13.9360,NaN,11.2000,...,14.09700,14.1410,14.5110,NaN,13.4110,13.6480,10.4210,NaN,12.5100,14.3190
SR8278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.5900,NaN,NaN,NaN
elocalcitol,13.2240,13.89500,12.8410,14.1290,16.7990,NaN,14.0190,13.7150,NaN,13.5060,...,12.62300,14.0710,16.8270,NaN,14.1680,14.5640,13.6430,10.8600,13.4250,13.7930
BRD-A05715709,NaN,14.99900,NaN,NaN,15.4380,NaN,NaN,14.9750,NaN,NaN,...,NaN,NaN,15.1590,NaN,14.0260,NaN,14.0740,14.0580,NaN,NaN
docetaxel,1.8294,1.06230,NaN,NaN,17.3740,NaN,NaN,4.8266,NaN,4.6722,...,NaN,NaN,NaN,NaN,5.8636,NaN,6.2694,10.8010,NaN,NaN
cyclophosphamide,NaN,NaN,14.7580,14.6580,NaN,14.2840,15.2040,NaN,NaN,NaN,...,15.15700,15.0630,NaN,15.1750,NaN,14.8030,NaN,NaN,14.9180,14.8600
3-Cl-AHPC,8.4779,7.23390,9.9731,NaN,13.5880,8.0256,9.4961,9.5906,9.9394,9.2713,...,9.97490,13.7790,12.7050,NaN,10.5630,9.7144,10.7600,10.2110,10.3690,10.0680


In [4]:
ref

A101D_SKIN                          1.681314e+02
A172_CENTRAL_NERVOUS_SYSTEM         1.297258e+02
A204_SOFT_TISSUE                    3.774041e+02
A2058_SKIN                          3.627115e+02
A2780_OVARY                         9.369074e-01
A375_SKIN                           1.587242e+02
A498_KIDNEY                         1.194004e+02
A549_LUNG                           2.098014e-04
A673_BONE                           4.952262e-03
A704_KIDNEY                         8.571483e-01
ABC1_LUNG                           3.225838e+03
ACCMESO1_PLEURA                     1.256594e+00
ACHN_KIDNEY                         3.729630e+02
AGS_STOMACH                         1.603415e+03
AM38_CENTRAL_NERVOUS_SYSTEM         1.867312e-09
AN3CA_ENDOMETRIUM                   1.465554e-02
ASPC1_PANCREAS                      1.772425e+02
AU565_BREAST                        4.874600e+03
BC3C_URINARY_TRACT                  1.234093e-08
BCPAP_THYROID                       4.625352e-02
BEN_LUNG            

In [5]:
features_random = ccal.support.make_random_features(10, 10)
ref_random = ccal.support.make_random_features(1, 10)

In [6]:
ref_random.index = random.sample(ref.index.tolist(), ref_random.size)

In [7]:
ref_random

HCC2279_LUNG                0.900964
NCIH2126_LUNG               0.953867
TE441T_SOFT_TISSUE          0.450822
NCIH510_LUNG                0.264426
NCIH441_LUNG                0.535814
HMC18_BREAST                0.726967
OSRC2_KIDNEY                0.285964
NCIH522_LUNG                0.347659
SKNBE2_AUTONOMIC_GANGLIA    0.882931
NCIH209_LUNG                0.474703
Name: Feature 0, dtype: float64

In [8]:
# ccal.analyze.rank_features_against_reference(features, ref_random, n_sampling=1, n_perm=1)

In [263]:
import math

import numpy as np
from scipy.stats import pearsonr
from statsmodels.nonparametric.kernel_density import KDEMultivariate
from scipy.stats import binom_test
# TODO: pythonize bcv
import rpy2.robjects as ro
from rpy2.robjects.numpy2ri import numpy2ri

ro.conversion.py2ri = numpy2ri
from rpy2.robjects.packages import importr

mass = importr('MASS')


def information_coefficient(x, y, z=None, n_grid=25, vector_data_type=None, n_perm=0, adaptive=True, alpha=0.05,
                            perm_alpha=0.05):
    """
    :param x: array-like, (n_samples,)
    :param y: array-like, (n_samples,)
    :param z: array-like, (n_samples,), optional, variable on which to condition
    :param n_grid: int, number of grid points at which to evaluate kernel density
    :param vector_data_types: str, 3 chars of 'c' (continuous), 'u' (unordered discrete), or 'o' (ordered discrete)
    :param n_perm: int, >0 will return a p-value in addition to the information coefficient
    :param adaptive: bool, quit permutations after achieving a specified confidence that the p-value is above (or below) alpha
    :param alpha: float, threshold empirical p-value for significance of IC
    :param perm_alpha: float, threshold probability for terminating adaptive permutation
    :return: float and float, information coefficient, and the empirical p-value if n_perm > 0
                Note that if adaptive, the accuracy of the empirical p-value will vary: values closer to alpha will be estimated
                more precisely, while values obviously greater or less than alpha will be estimated less precisely.
    """
    vectors = [x, y]
    if z:
        vectors.append(z)
        x, y, z = drop_nan_columns(vectors)
    else:
        x, y = drop_nan_columns(vectors)
        
    rho, p = pearsonr(x, y)
    print(rho, p)
    
    rho2 = abs(rho)
    bandwidth_scaling = 1 + (-0.75) * rho2
    
    mi = mutual_information(x, y, z=z, n_grid=n_grid,
                            vector_data_types=vector_data_type, bandwidth_scaling=bandwidth_scaling)
    ic_sign = np.sign(rho)
    ic = ic_sign * np.sqrt(1 - np.exp(- 2 * mi))

    if n_perm:
        n_more_extreme = 0
        trials = 0
        for i in range(n_perm):
            trials += 1
            # The question is whether I want to have
            # a certain width of confidence interval around my estimate of the pval
            # or just a certain confidence that the pval is greater than 0.05 (current solution)
            pm_x = np.random.permutation(x)
            pm_rho, p = pearsonr(pm_x, y)
            pm_rho2 = abs(pm_rho)
            pm_bandwidth_scaling = (1 + (-0.75) * pm_rho2)
            pm_mi = mutual_information(pm_x, y, z, n_grid=n_grid,
                                       vector_data_types=vector_data_type, bandwidth_scaling=pm_bandwidth_scaling)
            pm_ic_sign = np.sign(pm_rho)
            pm_ic = pm_ic_sign * np.sqrt(1 - np.exp(- 2 * pm_mi))
            if (pm_ic <= ic and ic < 0) or (0 < ic and ic <= pm_ic):
                n_more_extreme += 1
            if adaptive:
                ge_binom_p = binom_test(n_more_extreme, i + 1, alpha, alternative='greater')
                # * 2 because what I'm doing is two-sided testing in both directions
                if ge_binom_p * 2 < perm_alpha:
                    break
                le_binom_p = binom_test(n_more_extreme, i + 1, alpha, alternative='less')
                if le_binom_p * 2 < perm_alpha:
                    break
        p_value = n_more_extreme / float(trials)
        return ic, p_value
    else:
        return ic


# TODO: understand the math
def mutual_information(x, y, z=None, n_grid=25, vector_data_types=None, bandwidth_scaling=None):
    """
    :param x: array-like, (n_samples,)
    :param y: array-like, (n_samples,)
    :param z: array-like, (n_samples,), optional, variable on which to condition
    :param n_grid: int, number of grid points at which to evaluate kernel density
    :param vector_data_types: str, 3 chars of 'c' (continuous), 'u' (unordered discrete), or 'o' (ordered discrete)
    :param bandwidth_scaling: float,
    :return: float, information coefficient
    """
    vectors = [x, y]
    if z:
        vectors.append(z)
        x, y, z = drop_nan_columns(vectors)
    else:
        x, y = drop_nan_columns(vectors)
        
    if not vector_data_types:
        # TODO: guess variable types
        vector_data_types = 'c' * len(vectors)
    elif len(vector_data_types) is not len(vectors):
        raise ValueError('Number of specified variable types does not match number of vectors.')

    # Keep only columns that are not NaN in all vectors, and add jitter to the filtered vectors
    not_nan_filter = [True] * vectors[0].size
    for v in vectors:
        not_nan_filter &= ~np.isnan(v)
    if not_nan_filter.sum() < 3:
        return 0
    else:
        for i in range(len(vectors)):
            vectors[i] = vectors[i][not_nan_filter]
            vectors[i] += np.random.random_sample(vectors[i].size) * 1E-10
    print(vectors)

    grids = [np.linspace(v.min(), v.max(), n_grid) for v in vectors]
    print('grids', grids)
    mesh_grids = np.meshgrid(*grids)
    grid_shape = tuple([n_grid] * len(vectors))
    print(grid_shape)
    grid = np.vstack([mesh_grid.flatten() for mesh_grid in mesh_grids])
    print('grid', grid)
    delta = np.array([rbcv(q) for q in vectors]).reshape((len(vectors),)) / 4
    print('delta', delta)
    if bandwidth_scaling:
        delta *= bandwidth_scaling
        print('delta', delta)
    kde = KDEMultivariate(vectors, bw=delta, var_type=vector_data_types)
    p_joint = kde.pdf(grid).reshape(grid_shape) + np.finfo(float).eps
    print('p_joint', p_joint)
    ds = [grid[1] - grid[0] for grid in grids]
    ds_prod = np.prod(ds)
    p_joint /= (p_joint.sum() * ds_prod)
    h_joint = - np.sum(p_joint * np.log(p_joint)) * ds_prod
    dx = ds[0]
    dy = ds[1]
    if z:
        dz = ds[2]
        pxz = p_joint.sum(axis=1) * dy
        pyz = p_joint.sum(axis=0) * dx
        pz = p_joint.sum(axis=(0, 1)) * dx * dy
        hxz = -np.sum(pxz * np.log(pxz)) * dx * dz
        hyz = -np.sum(pyz * np.log(pyz)) * dy * dz
        hz = -np.sum(pz * np.log(pz)) * dz
        cmi = hxz + hyz - h_joint - hz
        return cmi
    else:
        dx = ds[0]
        print('dx', dx)
        dy = ds[1]
        print('dy', dy)
        px = p_joint.sum(axis=1) * dy
        print(px)
        py = p_joint.sum(axis=0) * dx
        print(py)
        hx = -np.sum(px * np.log(px)) * dx
        hy = -np.sum(py * np.log(py)) * dy
        mi = hx + hy - h_joint
        print('XX', mi)
        return mi


def rbcv(x):
    """
    :param x: array-like, (n_samples,)
    :return: float, bandwidth
    """
    bandwidth = np.array(mass.bcv(x))[0]
    return bandwidth


def drop_nan_columns(vectors):
    """
    Keep only column positions that are not nan in all vectors.
    :param vectors: list of numpy array, must have the same length (avoid [v1, ..., vn])
    :return: list of numpy arrays,
    """
    for v in vectors[1:]:
        if len(v) != len(vectors[0]):
            raise ValueError('Input arrays have different lengths.')
            
    not_nan_filter = [True] * len(vectors[0])
    for v in vectors:
        not_nan_filter &= ~np.isnan(v)
    
    only_not_nan = []
    for i in range(len(vectors)):
        only_not_nan.append(vectors[i][not_nan_filter])
    return only_not_nan
    
    
def add_jitter(vectors, inplace=False):
    """
    Add jitter to vectors inplace.
    :param vectors: numpy array,
    :return: None
    """
    for i in range(len(vectors)):
        vectors[i] += np.random.random_sample(vectors[i].size) * 1E-0

In [265]:
x = np.array([12.517, 14.706, np.nan, 14.12, np.nan, np.nan, np.nan, 12.255])
y = np.array([0.98246356, 0.97525171, 0.77744759, 0.64084311, 0.4405853, 0.43827196, 0.12447757, 0.08116039])
xx = np.random.random_sample(7)
xx[3] = None
yy = np.random.random_sample(7)
yy[5]=None
information_coefficient(x, y)

0.509123886951 0.490876113049
[array([ 12.517,  14.706,  14.12 ,  12.255]), array([ 0.98246356,  0.97525171,  0.64084311,  0.08116039])]
grids [array([ 12.255   ,  12.357125,  12.45925 ,  12.561375,  12.6635  ,
        12.765625,  12.86775 ,  12.969875,  13.072   ,  13.174125,
        13.27625 ,  13.378375,  13.4805  ,  13.582625,  13.68475 ,
        13.786875,  13.889   ,  13.991125,  14.09325 ,  14.195375,
        14.2975  ,  14.399625,  14.50175 ,  14.603875,  14.706   ]), array([ 0.08116039,  0.11871469,  0.15626899,  0.19382329,  0.23137759,
        0.26893188,  0.30648618,  0.34404048,  0.38159478,  0.41914908,
        0.45670338,  0.49425768,  0.53181198,  0.56936627,  0.60692057,
        0.64447487,  0.68202917,  0.71958347,  0.75713777,  0.79469207,
        0.83224637,  0.86980066,  0.90735496,  0.94490926,  0.98246356])]
(25, 25)
grid [[ 12.255       12.357125    12.45925    ...,  14.50175     14.603875
   14.706     ]
 [  0.08116039   0.08116039   0.08116039 ...,   0.9824635

0.35591890867684606